In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.externals.joblib import dump, load
from tensorflow.keras import layers, models
import numpy as np
import os

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
# Loading numpy files
X = np.load(os.path.join("..", "IPD", "data_numpy_new.npy"))
y = np.load(os.path.join("..", "IPD", "label_numpy_new.npy"))
print((X.shape))
print(y.shape)

(415, 232, 12)
(415,)


In [4]:
#preparing input data for CNN classifier

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
scaler = StandardScaler()

# Scale train data
# print(x_train.shape)
num_instances, num_time_steps, num_features = x_train.shape
# print(num_features)
# print(num_instances, num_time_steps, num_features)
x_train = np.reshape(x_train, (-1, num_features))
x_train = scaler.fit_transform(x_train)
# x_train_new = scaler_new.fit_transform(x_train)
x_train = np.reshape(x_train, (num_instances, num_time_steps, num_features))
# print(num_instances, num_time_steps, num_features)
# Scale test data
num_instances, num_time_steps, num_features = x_test.shape
x_test = np.reshape(x_test, (-1, num_features))
x_test = scaler.transform(x_test)
# x_test_new = scaler_new.transform(x_test)
x_test = np.reshape(x_test, (num_instances, num_time_steps, num_features))
x_train_lstm=x_train
# x_test_lstm=x_test
dump(scaler, 'std_scaler.bin', compress=True)
x_train = np.expand_dims(x_train, axis = 3)
x_test = np.expand_dims(x_test, axis = 3)
# print(x_test.shape)

In [109]:
# Define the model

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(232, num_features, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding="same"))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu', padding="same"))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu', padding="same"))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(6, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],)

history = model.fit(x_train, y_train, epochs=50, 
                    validation_data=(x_test, y_test))

Train on 373 samples, validate on 42 samples
Epoch 1/50
373/373 [==============================] - 3s 7ms/sample - loss: 1.8025 - accuracy: 0.1635 - val_loss: 1.7944 - val_accuracy: 0.1429
Epoch 2/50
373/373 [==============================] - 1s 3ms/sample - loss: 1.7881 - accuracy: 0.1716 - val_loss: 1.8047 - val_accuracy: 0.1190
Epoch 3/50
373/373 [==============================] - 1s 3ms/sample - loss: 1.7547 - accuracy: 0.2627 - val_loss: 1.7844 - val_accuracy: 0.1667
Epoch 4/50
373/373 [==============================] - 1s 4ms/sample - loss: 1.6630 - accuracy: 0.2708 - val_loss: 1.7505 - val_accuracy: 0.2381
Epoch 5/50
373/373 [==============================] - 1s 4ms/sample - loss: 1.6231 - accuracy: 0.3271 - val_loss: 1.6144 - val_accuracy: 0.2619
Epoch 6/50
373/373 [==============================] - 1s 4ms/sample - loss: 1.4757 - accuracy: 0.3807 - val_loss: 1.5154 - val_accuracy: 0.3571
Epoch 7/50
373/373 [==============================] - 1s 4ms/sample - loss: 1.3556 - accura

In [17]:
print(x_train.shape)

(373, 232, 12, 1)


In [18]:
print(y_train.shape)

(373,)


In [79]:
scores = model.evaluate(x_test, y_test, verbose=0)
print(scores)
print("Accuracy: %.2f%%" % (scores[1]*100))

[1.770480065118699, 0.52380955]
Accuracy: 52.38%


In [77]:
#LSTM model(ignore for now)
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
model_s = tf.keras.Sequential()

model_s.add(LSTM(100, input_shape=(232, num_features)))
# model_s.add(Dropout(0.5))
model_s.add(Dense(100, activation='relu'))
model_s.add(Dense(6, activation='softmax'))
model_s.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model_s.fit(x_train_lstm, y_train, epochs=50, validation_data=(x_test_lstm, y_test))

Train on 373 samples, validate on 42 samples
Epoch 1/50
373/373 [==============================] - 10s 28ms/sample - loss: 1.7938 - accuracy: 0.1689 - val_loss: 1.7908 - val_accuracy: 0.1667
Epoch 2/50
373/373 [==============================] - 3s 9ms/sample - loss: 1.7918 - accuracy: 0.1716 - val_loss: 1.7930 - val_accuracy: 0.1667
Epoch 3/50
373/373 [==============================] - 4s 10ms/sample - loss: 1.7903 - accuracy: 0.1743 - val_loss: 1.7938 - val_accuracy: 0.1667
Epoch 4/50
373/373 [==============================] - 4s 11ms/sample - loss: 1.7890 - accuracy: 0.1716 - val_loss: 1.7932 - val_accuracy: 0.1429
Epoch 5/50
373/373 [==============================] - 4s 12ms/sample - loss: 1.7919 - accuracy: 0.1662 - val_loss: 1.7931 - val_accuracy: 0.1190
Epoch 6/50
373/373 [==============================] - 4s 9ms/sample - loss: 1.7896 - accuracy: 0.1394 - val_loss: 1.7943 - val_accuracy: 0.1190
Epoch 7/50
373/373 [==============================] - 4s 10ms/sample - loss: 1.7897 - 

KeyboardInterrupt: 

In [116]:
#KNN Classifier
from sklearn.model_selection import GridSearchCV
print(x_train.shape)
print(num_instances)
from sklearn.neighbors import KNeighborsClassifier
x_train_knn = np.reshape(x_train,(373, num_time_steps*num_features))
x_test_knn = np.reshape(x_test,(42, num_time_steps*num_features))
print(x_train_knn.shape)
n_neighbors = list(range(3,30))
p=[1,2]
#Convert to dictionary
hyperparameters = dict(n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)
#Fit the model
best_model = clf.fit(x_train_knn,y_train)
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])
print(best_model.score(x_test_knn, y_test))

(373, 232, 12, 1)
42
(373, 2784)
Best p: 1
Best n_neighbors: 3
0.6190476190476191


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [15]:
#Random Forest Classifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
hyperparameters = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80],
 'max_features': ['auto', 'sqrt'],
#  'min_samples_leaf': [1, 2, 4],
#  'min_samples_split': [2, 5, 10],
 'n_estimators': [1, 2, 4, 8]}
x_train_knn = np.reshape(x_train,(373, num_time_steps*num_features))
x_test_knn = np.reshape(x_test,(42, num_time_steps*num_features))
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = GridSearchCV(rf, hyperparameters, cv=2)
# Fit the random search model
rf_random.fit(x_train_knn, y_train)
print(rf_random.best_params_)
print(rf_random.score(x_train_knn, y_train))
print(rf_random.score(x_test_knn, y_test))

{'bootstrap': False, 'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 8}
1.0
0.7857142857142857


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [129]:

model = models.Sequential()
model.add(layers.TimeDistributed(layers.Conv2D(64, (3, 3)),
                          input_shape=(232,num_features)))
model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))
model.add(layers.TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu')))
model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))
model.add(layers.TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu', padding="same")))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu', padding="same"))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu', padding="same"))
model.add(layers.TimeDistributed(layers.Flatten()))
model.add(layers.TimeDistributed(layers.Dense(64, activation='relu')))
model.add(layers.TimeDistributed(layers.Dense(6, activation='softmax')))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],)

history = model.fit(x_train_lstm, y_train, epochs=50, 
                    validation_data=(x_test, y_test))


ValueError: input tensor must have rank 4